In [71]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

import zoopla_clean as zc
import zoopla_scrape as zs

import importlib
importlib.reload(zc)
importlib.reload(zs)

to_scrape_df = pd.read_csv("to_scrape.csv")

to_scrape_df



,location,link
0,manchester,https://www.zoopla.co.uk/for-sale/property/man...
1,salford,https://www.zoopla.co.uk/for-sale/property/gre...
2,bolton,https://www.zoopla.co.uk/for-sale/property/gre...
3,wigan,https://www.zoopla.co.uk/for-sale/property/wig...
4,stockport,https://www.zoopla.co.uk/for-sale/property/sto...
5,bury,https://www.zoopla.co.uk/for-sale/property/gre...
6,rochdale,https://www.zoopla.co.uk/for-sale/property/roc...
7,oldham,https://www.zoopla.co.uk/for-sale/property/old...
8,tameside,https://www.zoopla.co.uk/for-sale/property/tam...


In [26]:
df = pd.DataFrame()
data_frames = []

urls = to_scrape_df['link'].to_list()

for url in urls:
    data_frames.append(zs.get_house_data(url))

Invalid page 37
Error count: 43
Error count: 5
Error count: 4
Error count: 2
Error count: 1
Error count: 0
Error count: 3
Error count: 3
Error count: 0


In [38]:
df = pd.concat(data_frames)

size_start = (len(df))
df = zc.clean_type(df)
df = zc.clean_price(df)
df = zc.get_dist_to_station(df)
df = zc.clean_address(df)
df = zc.remove_outliers(df)
size_end = (len(df))

print(f"Database of {size_start} reduced to {size_end} after cleaning")
df.head(4)

Database of 5489 reduced to 4397 after cleaning


,type,price,station,beds,baths,receptions,post_code,location
0,flat,369950.0,0.5,1,1.0,0.0,M15,manchester
1,semi-detached,175000.0,0.1,2,1.0,1.0,M12,manchester
2,flat,270000.0,0.0,1,1.0,1.0,M3,salford
3,flat,270000.0,0.9,2,1.0,1.0,M20,manchester


In [72]:
average_list = zs.get_post_code_data(df)
average_list

65 unique post codes


,post_code,avg_sold_price_12months
0,BL6,243088.0
1,SK2,263129.0
2,M5,183046.0
3,WN2,150601.0
4,WN1,200168.0
...,...,...
60,M30,224262.0
61,M40,163603.0
62,BL3,154844.0
63,M14,195420.0


In [79]:
df2 = pd.merge(df,average_list,left_on='post_code',right_on='post_code')
df2 = df2[df2['avg_sold_price_12months'].notna()]
df2 = df2.reset_index(drop=True)
df2

TypeError: reset_index() got an unexpected keyword argument 'keep'

,index,type,price,station,beds,baths,receptions,post_code,location,avg_sold_price_12months
0,0,flat,369950.0,0.5,1,1.0,0.0,M15,manchester,200730.0
1,1,flat,369950.0,0.5,1,1.0,0.0,M15,manchester,200730.0
2,2,flat,154950.0,0.5,2,1.0,1.0,M15,manchester,200730.0
3,3,flat,541805.0,0.2,3,1.0,1.0,M15,manchester,200730.0
4,4,flat,240327.0,0.2,1,1.0,1.0,M15,manchester,200730.0
...,...,...,...,...,...,...,...,...,...,...
4371,4392,detached,625000.0,0.7,3,2.0,3.0,M34,manchester,193433.0
4372,4393,semi-detached,230000.0,0.7,3,1.0,2.0,M34,manchester,193433.0
4373,4394,semi-detached,310000.0,0.6,4,1.0,0.0,M34,manchester,193433.0
4374,4395,terraced,130000.0,0.2,3,1.0,1.0,M34,manchester,193433.0


In [75]:
df2.to_csv("zoopla_data.csv",index=False)